In [2]:
import pandas as pd
import os
from os import listdir, rename
from os.path import isfile, join, exists
from shutil import copyfile

In [3]:
img_path = r'D:\Coding\Python\jupyter\Cards'

In [4]:
data = pd.read_excel(r'D:\Coding\Python\jupyter\Cards\docs\_CatalogMaster.xlsx', dtype={'ISBN': str})

In [5]:
df = pd.DataFrame(data, columns=['ISBN', 'ProductCode']).dropna(how='all')

In [6]:
img_files_list = [f for f in listdir(img_path) if isfile(join(img_path, f))]

In [7]:
def try_get_isbn(product_code):
    try:
        line = df.loc[df['ProductCode'] == product_code]
        isbn = (True, line.iloc[0,0])
    except:
        isbn = (False, 'Product Code not found')  
    return isbn

In [9]:
for image in img_files_list:
    img_product_code = image.split("-")[0]
    img_sub_list = [f for f in listdir(img_path) if img_product_code == f.split("-")[0]]
    
    for img in img_sub_list:
        isbn = try_get_isbn(img_product_code)
        if isbn[0]:
            if exists(join(img_path, f"{isbn[1]}.jpg")):
                rename(join(img_path, img), join(img_path, f"{isbn[1]}_1.jpg"))
            else:
                rename(join(img_path, img), join(img_path, f"{isbn[1]}.jpg"))
        